In [36]:
import numpy as np

In [37]:
import pandas as pd

In [38]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [39]:
data=pd.read_csv('train.csv')
data

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
21007,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
21008,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
21009,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
21010,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [40]:
data.shape

(21012, 5)

In [41]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [42]:
data.isnull()

,id,title,author,text,label
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
21007,False,False,False,False,False
21008,False,False,False,False,False
21009,False,False,False,False,False
21010,False,False,False,False,False


In [43]:
data.isnull().sum()

id           0
title      565
author    1974
text        39
label        0
dtype: int64

In [44]:
data=data.fillna('')

In [45]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [46]:
data['main']=data['author']+' '+data['title']

In [47]:
print(data['main'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
21007    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
21008    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
21009    Michael J. de la Merced and Rachel Abrams Macy...
21010    Alex Ansary NATO, Russia To Hold Parallel Exer...
21011              David Swanson What Keeps the F-35 Alive
Name: main, Length: 21012, dtype: object


In [48]:
# seperating data and label
X=data.drop(columns='label',axis=1)
y=data['label']

In [49]:
#stemming is the process to reduce a particular word to it's root word
port_stem=PorterStemmer()

In [50]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [52]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [53]:
data['main']=data['main'].apply(stemming)

In [54]:
print(data['main'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
21007    jerom hudson rapper trump poster child white s...
21008    benjamin hoffman n f l playoff schedul matchup...
21009    michael j de la merc rachel abram maci said re...
21010    alex ansari nato russia hold parallel exercis ...
21011                            david swanson keep f aliv
Name: main, Length: 21012, dtype: object


In [55]:
#seperating main and label
X=data['main'].values
Y=data['label'].values

In [56]:
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [57]:
Y

array([1, 0, 1, ..., 0, 1, 1])

In [58]:
Y.shape

(21012,)

In [59]:
# converting textual data to numerical data
vectorizer=TfidfVectorizer() #it counts a particular word  is repeating number of times in text or paragraph
vectorizer.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [60]:
X=vectorizer.transform(X)

In [62]:
print(X)

  (0, 15686)	0.2853146701481354
  (0, 13473)	0.257595477535434
  (0, 8909)	0.36113056744993804
  (0, 8630)	0.2932103564379448
  (0, 7692)	0.2488384804456875
  (0, 7005)	0.21953427980052698
  (0, 4973)	0.2337297940747089
  (0, 3792)	0.26986412582319796
  (0, 3600)	0.3577170220034746
  (0, 2959)	0.2473331517680974
  (0, 2483)	0.3689059530918693
  (0, 267)	0.2702842922897975
  (1, 16799)	0.300227712410275
  (1, 6816)	0.19031945476997492
  (1, 5503)	0.7146731077518684
  (1, 3568)	0.26365453312409093
  (1, 2813)	0.19075444229007987
  (1, 2223)	0.38287961136710974
  (1, 1894)	0.154936542628728
  (1, 1497)	0.29390657658614017
  (2, 15611)	0.41618665583713965
  (2, 9620)	0.49138895894783263
  (2, 5968)	0.3484576624399318
  (2, 5389)	0.3871750297474775
  (2, 3103)	0.4612347285056164
  :	:
  (21009, 13122)	0.24849022037837726
  (21009, 12344)	0.27196798207065986
  (21009, 12138)	0.24709802841895384
  (21009, 10306)	0.08039385014463835
  (21009, 9588)	0.17416621397132473
  (21009, 9518)	0.2956302

In [64]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [65]:
model=LogisticRegression()

In [66]:
model.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [71]:
Y_pred=model.predict(X_train)
acc=accuracy_score(Y_pred,Y_train)

In [72]:
acc

0.9870902492712238